In [2]:
import tensorflow as tf

# GPU 디바이스 확인
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# TensorFlow가 GPU를 사용할 수 있도록 설정합니다.
if len(tf.config.list_physical_devices('GPU')) > 0:
    print('GPU is enabled')
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)


Num GPUs Available:  1
GPU is enabled


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import matplotlib.pyplot as plt

# Error #15: Initializing libiomp5md.dll, but found libiomp5 already initialized. 해결
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# 디렉토리 지정
base_dir = '/content/drive/MyDrive/ColabNotebooks/프로젝트/PneumoniaClassifier/data'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# 기본값 설정
target_size = (224, 224)

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

batch_size = 64

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    batch_size=1,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=1,
    class_mode='binary')

# 클래스 불균형 해결을 위한 class_weight 계산
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_generator.classes),
                                        y = train_generator.classes                                                   
                                    )
class_weights = dict(zip(np.unique(train_generator.classes), class_weights))
class_weights


In [4]:
import tensorflow as tf

In [5]:
def dir_file_count(directory):
    return sum([len(files) for r, d, files in os.walk(directory)])

In [7]:
# batch_size = 32

# target_size = (299, 299)
# color_mode = "rgb"


rescale = 1./255
target_size = (128,128)
batch_size = 64
# class_mode = "categorical"
class_mode = "binary"


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle=True)


validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=rescale)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=dir_file_count(validation_dir),
    shuffle = False)


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=rescale)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=dir_file_count(test_dir),
    shuffle = False)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [11]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import mobilenet_v2

# ResNet50 pre-trained on imagenet
# base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(128,128, 3))
base_model = mobilenet_v2.MobileNetV2(include_top=False, weights='imagenet', input_shape=(128,128, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# 마지막 5개의 합성곱 레이어를 동결해제
for layer in base_model.layers[:-5]:
    layer.trainable = True

# Add classification layers
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1))(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Build model
model = tf.keras.Model(inputs=base_model.input, outputs=output)

9406464/9406464 [==============================] - 0s 0us/step


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten

def get_conv_model():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=(128,128,3)))
    model.add(Conv2D(16, (3, 3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    '''
    model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=(128,128,3)))
    model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
    model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
    model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
    model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    '''

    model.add(Flatten())

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(1 , activation='sigmoid'))


    print(model.summary())
    
    return model



In [12]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)

model_path = './model.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

In [ ]:
model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 모델을 훈련합니다
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    epochs=20,
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    verbose=1,
                    callbacks=[checkpoint, early_stopping],
                    shuffle=True
                    )

Epoch 1/20
 8/81 [=>............................] - ETA: 11:49 - loss: 37.6329 - accuracy: 0.8105

In [ ]:
# test
import numpy as np
from sklearn.metrics import classification_report

# test
predictions = model.predict(test_generator)
y_pred = np.round(predictions)
y_true = test_generator.classes

print(classification_report(y_true, y_pred))

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'go', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('loss')
plt.legend()

plt.show()

model.save('model.h5')
